# Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data Loading

In [3]:
TruthFrequencyEvents_df = pd.read_csv('TruthFrequencyEvents.txt',sep='\t', names = ["data"])
FrequencyEvents_df = pd.read_csv('FrequencyEvents.txt',sep='\t', names = ["data"])
TruthBearingEvents_df = pd.read_csv('TruthBearingEvents.txt',sep='\t', names = ["data"])
BearingEvents_df = pd.read_csv('BearingEvents.txt',sep='\t', names = ["data"])
TargetPositions_df = pd.read_csv('TargetPositions.txt',sep='\t', names = ["data"])

# Data Preprocessing

In [4]:
TruthBearingEvents=[]
for _, row in TruthBearingEvents_df.iterrows():
  temp = row["data"].split()
  temp[0] = float(temp[0])
  temp[1] = float(temp[1])
  temp[2] = float(temp[2])
  if not(temp[0]==0 and temp[1]==0 and temp[2]==0):
    TruthBearingEvents.append(temp)

In [5]:
TruthFrequencyEvents=[]
for _, row in TruthFrequencyEvents_df.iterrows():
  temp = row["data"].split()
  temp[0] = float(temp[0])
  temp[1] = float(temp[1])
  if not(temp[0]==0 and temp[1]==0 and temp[2]=='0'):
    TruthFrequencyEvents.append(temp)
# SNR is not converted to float because of missing values

In [6]:
TargetPositions=[]
for _, row in TargetPositions_df.iterrows():
  temp = row["data"].split()
  for i in range(len(temp)):
    temp[i]=float(temp[i])
  TargetPositions.append(temp)

In [7]:
T1TruthBearing = []
T2TruthBearing = []
T3TruthBearing = []
T4TruthBearing = []
OTruthBearing = []
i=0
while i<len(TruthBearingEvents):
  T1TruthBearing.append(TruthBearingEvents[i])
  T2TruthBearing.append(TruthBearingEvents[i+1])
  T3TruthBearing.append(TruthBearingEvents[i+2])
  T4TruthBearing.append(TruthBearingEvents[i+3])
  OTruthBearing.append(TruthBearingEvents[i+4])
  i=i+5

In [8]:
T1TruthFrequency = []
T2TruthFrequency = []
T3TruthFrequency = []
T4TruthFrequency = []
i=0
while i<len(TruthFrequencyEvents):
  T1TruthFrequency.append(TruthFrequencyEvents[i])
  T1TruthFrequency.append(TruthFrequencyEvents[i+1])
  T1TruthFrequency.append(TruthFrequencyEvents[i+2])
  T1TruthFrequency.append(TruthFrequencyEvents[i+3])
  T2TruthFrequency.append(TruthFrequencyEvents[i+4])
  T2TruthFrequency.append(TruthFrequencyEvents[i+5])
  T2TruthFrequency.append(TruthFrequencyEvents[i+6])
  T2TruthFrequency.append(TruthFrequencyEvents[i+7])
  T3TruthFrequency.append(TruthFrequencyEvents[i+8])
  T3TruthFrequency.append(TruthFrequencyEvents[i+9])
  T3TruthFrequency.append(TruthFrequencyEvents[i+10])
  T3TruthFrequency.append(TruthFrequencyEvents[i+11])
  T4TruthFrequency.append(TruthFrequencyEvents[i+12])
  T4TruthFrequency.append(TruthFrequencyEvents[i+13])
  T4TruthFrequency.append(TruthFrequencyEvents[i+14])
  T4TruthFrequency.append(TruthFrequencyEvents[i+15])
  i=i+16

In [9]:
T1Position = []
T2Position = []
T3Position = []
T4Position = []
OPosition = []
for i in range(len(TargetPositions)):
  T1Position.append([TargetPositions[i][0],TargetPositions[i][1],TargetPositions[i][2],TargetPositions[i][3]])
  T2Position.append([TargetPositions[i][0],TargetPositions[i][4],TargetPositions[i][5],TargetPositions[i][6]])
  T3Position.append([TargetPositions[i][0],TargetPositions[i][7],TargetPositions[i][8],TargetPositions[i][9]])
  T4Position.append([TargetPositions[i][0],TargetPositions[i][10],TargetPositions[i][11],TargetPositions[i][12]])
  OPosition.append([TargetPositions[i][0],TargetPositions[i][13],TargetPositions[i][14],TargetPositions[i][15]])

In [10]:
#Replace NaN values in truth frequency with Mean
t = 0
for i in range(len(T4TruthFrequency)):
  x = (T4TruthFrequency[i][2:3])[0]
  if(x!='-1.#INF'):
    t=t+float(x)
#mean
mean_t = t/len(T4TruthFrequency)
for i in T4TruthFrequency:
  if i[2]!='-1.#INF':
    i[2]=float(i[2])
  else:
    i[2]=mean_t

In [11]:
#Convert remaining Truth Frequency events to float
for i in T1TruthFrequency:
    i[2]=float(i[2])
for i in T2TruthFrequency:
    i[2]=float(i[2])
for i in T3TruthFrequency:
    i[2]=float(i[2])

# ANN for Target 1

In [0]:
#Dataset
X = []
for i in range(len(T1TruthFrequency)):
  y=[]
  y.append(T1TruthBearing[i][0],T1TruthBearing[i][1],T1TruthBearing[i][2], )